In [2]:
#installing transformers
!pip install transformers

In [3]:
##import and load the model
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [4]:
##load summarization pipeline
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [6]:
#URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
#URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
URL = "https://www.javatpoint.com/machine-learning"
r = requests.get(URL)

In [7]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [8]:
ARTICLE

'Machine Learning Tutorial Machine Learning tutorial provides basic and advanced concepts of machine learning. Our machine learning tutorial is designed for students and working professionals. Machine learning is a growing technology which enables computers to learn automatically from past data. Machine learning uses various algorithms for building mathematical models and making predictions using historical data or information. Currently, it is being used for various tasks such as image recognition, speech recognition, email filtering, Facebook auto-tagging, recommender system, and many more. This machine learning tutorial gives you an introduction to machine learning along with the wide range of machine learning techniques such as Supervised, Unsupervised, and Reinforcement learning. You will learn about regression and classification models, clustering methods, hidden Markov models, and various sequential models. In the real world, we are surrounded by humans who can learn everything 

In [9]:
## Chunk text
max_chunk = 500

In [10]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [11]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [12]:
len(chunks)

3

In [13]:
##summarize text

res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)



In [14]:
res[0]

{'summary_text': ' Machine Learning tutorial provides basic and advanced concepts of machine learning . Machine learning is a growing technology which enables computers to learn automatically from past data . Currently, it is being used for various tasks such as image recognition, speech recognition, email filtering, Facebook auto-tagging, recommender system, and many more .'}

In [15]:
' '.join([summ['summary_text'] for summ in res])


' Machine Learning tutorial provides basic and advanced concepts of machine learning . Machine learning is a growing technology which enables computers to learn automatically from past data . Currently, it is being used for various tasks such as image recognition, speech recognition, email filtering, Facebook auto-tagging, recommender system, and many more .  Machine learning is used in self-driving cars, cyber fraud detection, face recognition, and friend suggestion by Facebook . The performance of the machine learning algorithm depends on the amount of data, and it can be determined by the cost function . With the help of machine learning, we can save both time and money .  Machine learning is making our day to day life easy from self-driving cars to Amazon virtual assistant "Alexa" The goal of an agent is to get the most reward points, and hence, it improves its performance . Modern machine learning models can be used for making various predictions, including weather prediction, dis

In [16]:
text = ' '.join([summ['summary_text'] for summ in res])


In [17]:
text

' Machine Learning tutorial provides basic and advanced concepts of machine learning . Machine learning is a growing technology which enables computers to learn automatically from past data . Currently, it is being used for various tasks such as image recognition, speech recognition, email filtering, Facebook auto-tagging, recommender system, and many more .  Machine learning is used in self-driving cars, cyber fraud detection, face recognition, and friend suggestion by Facebook . The performance of the machine learning algorithm depends on the amount of data, and it can be determined by the cost function . With the help of machine learning, we can save both time and money .  Machine learning is making our day to day life easy from self-driving cars to Amazon virtual assistant "Alexa" The goal of an agent is to get the most reward points, and hence, it improves its performance . Modern machine learning models can be used for making various predictions, including weather prediction, dis

In [18]:
## output to text file
with open('blogsummary.text','w') as f:
    f.write(text)

In [19]:
pip install Rouge

Note: you may need to restart the kernel to use updated packages.


In [20]:
from rouge import Rouge
r = Rouge()

In [21]:
r.get_scores(ARTICLE,text)

[{'rouge-1': {'r': 0.9904761904761905,
   'p': 0.1793103448275862,
   'f': 0.3036496324407268},
  'rouge-2': {'r': 0.9315068493150684,
   'p': 0.11724137931034483,
   'f': 0.2082695232821071},
  'rouge-l': {'r': 0.9904761904761905,
   'p': 0.1793103448275862,
   'f': 0.3036496324407268}}]

In [22]:
import pickle as pkl

In [24]:
pkl.dump(summarizer,open("text_summarizer.pkl","wb"))